In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
df = pd.read_csv(r"C:\DATA\data_4.1.csv")
df['BMI'] = df['weight_kg'] / (df['height_cm']/100)**2
def bmi_class(bmi):
    if bmi < 18.5: return 'underweight'
    elif bmi < 25: return 'normal'
    else: return 'overweight'
df['BMI_class'] = df['BMI'].apply(bmi_class)

# Encode target
label_map = {'underweight':0, 'normal':1, 'overweight':2}
y = df['BMI_class'].map(label_map).values

# Preprocess X
X = df[['job','age','height_cm','weight_kg']]
cat = OneHotEncoder(sparse_output=False)
jobs_enc = cat.fit_transform(X[['job']])
scaler = StandardScaler()
nums = scaler.fit_transform(X[['age','height_cm','weight_kg']])
X_enc = np.hstack([jobs_enc, nums])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42, stratify=y)

# Reshape to 3D (samples, timesteps, features=1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test  = X_test.reshape(X_test.shape[0],  X_test.shape[1], 1)

# Function to build model
def build_cnn(input_shape):
    model = models.Sequential([
        layers.Conv1D(32, 2, activation='relu', input_shape=input_shape),
        layers.Conv1D(64, 2, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_rnn(input_shape):
    model = models.Sequential([
        layers.SimpleRNN(64, return_sequences=True, input_shape=input_shape),
        layers.SimpleRNN(32),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = models.Sequential([
        layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        layers.LSTM(32),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train each model (CNN as example)
cnn = build_cnn((X_train.shape[1],1))
cnn.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

rnn = build_rnn((X_train.shape[1],1))
rnn.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

lstm = build_lstm((X_train.shape[1],1))
lstm.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
cnn_acc = cnn.evaluate(X_test, y_test, verbose=0)[1]
rnn_acc = rnn.evaluate(X_test, y_test, verbose=0)[1]
lstm_acc = lstm.evaluate(X_test, y_test, verbose=0)[1]

print("CNN accuracy:", cnn_acc)
print("RNN accuracy:", rnn_acc)
print("LSTM accuracy:", lstm_acc)





Epoch 1/20


C:\Users\longpt1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5516 - loss: 0.8842 - val_accuracy: 0.6438 - val_loss: 0.7172
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6859 - loss: 0.6492 - val_accuracy: 0.6313 - val_loss: 0.6807
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6984 - loss: 0.6144 - val_accuracy: 0.6938 - val_loss: 0.6586
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7125 - loss: 0.5924 - val_accuracy: 0.7000 - val_loss: 0.6327
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7078 - loss: 0.5766 - val_accuracy: 0.7125 - val_loss: 0.6186
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7250 - loss: 0.5588 - val_accuracy: 0.7188 - val_loss: 0.6216
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7406 - loss: 0.5539 - val_accuracy: 0.7250 - val_loss: 0.6139
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7422 - loss: 0.5421 - val_accuracy: 0.7250 - val_loss: 0.6275
E

C:\Users\longpt1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6031 - loss: 0.8396 - val_accuracy: 0.7312 - val_loss: 0.6533
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7516 - loss: 0.5723 - val_accuracy: 0.7625 - val_loss: 0.5627
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7859 - loss: 0.5079 - val_accuracy: 0.7750 - val_loss: 0.5092
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8156 - loss: 0.4610 - val_accuracy: 0.8000 - val_loss: 0.4494
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8422 - loss: 0.3943 - val_accuracy: 0.8313 - val_loss: 0.3741
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8734 - loss: 0.3547 - val_accuracy: 0.8750 - val_loss: 0.3232
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9031 - loss: 0.3049 - val_accuracy: 0.9250 - val_loss: 0.2598
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8969 - loss: 0.2772 - val_accuracy: 0.8938 - val_loss: 0.2583
Ep

TypeError: 'module' object is not subscriptable